In [ ]:
import pandas as pd
import numpy as np
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import os
os.environ['R_HOME'] = '/root/miniconda3/envs/R/lib/R'

import rpy2.robjects as objects
from rpy2.robjects.packages import importr
base = importr('base')
r_pROC = importr("pROC")
base._libPaths()[0]

def df_stats(df):
    from tabulate import tabulate
    print("\n***** Shape: ", df.shape," *****\n")
    
    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()
    
    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(list_stat_val, columns=['Name', 'Null', 'Unique', 'Dtypes'])
    print(tabulate(df_stat_val, headers='keys', tablefmt='psql'))
    return df.head()

In [ ]:
df_ifr = pd.read_csv('../CathAI/data/DeepCORO/CathReport_MHI_Merged/2017-2021_CathReport_first_value_in_interval_IFR.csv')

In [ ]:
## Define column where IFR was performed
import pandas as pd

def create_new_col(df):
    # List of column names to be checked
    col_names = ['D2', 'D3', 'RVG1', 'RVG2', 'S1', 'Saphène ou mammaire', 'bx', 'diagonal', 'dist_lad', 'dist_lcx', 
                 'dist_rca', 'lad', 'lcx', 'leftmain', 'lvp', 'marg_d', 'mid_lad', 'mid_rca', 'om1', 'om2', 'om3', 
                 'pda', 'posterolateral', 'prox_rca']
    
    # Iterating over rows using DataFrame.apply() and lambda function
    df['ifr_performed'] = df.apply(lambda row: ', '.join([col for col in col_names if row[col] != -1.0]), axis=1)
    return df


In [ ]:
df = create_new_col(df_ifr)
df['contains_lad'] = df['ifr_performed'].str.contains('lad')
df['contains_lcx'] = df['ifr_performed'].str.contains('lcx')
df['contains_rca'] = df['ifr_performed'].str.contains('rca|pda|posterolateral')


In [ ]:
display(df_ifr.head(n=1))

In [ ]:
### Merge with DICOMS Extracted

df_dicoms = pd.read_csv('../CathAI/data/DeepCORO/EXAMS_Extracted/2017-2021_dicom_extracted_mod.csv')

In [ ]:
# Identify common columns in df and df_dicoms, excluding 'path'
common_cols = [col for col in df.columns if col in df_dicoms.columns and col != 'path']

# Drop common columns from df_dicoms
df_dicoms_dropped = df_dicoms.drop(columns=common_cols)

# Perform left merge
df_merged = pd.merge(df, df_dicoms_dropped, left_on='DICOMPath', right_on='path', how='left')


In [ ]:
### Merge with views 
df_angle_object = pd.read_csv('../CathAI/data/DeepCORO/CATHAI_Extracted_Concatenated/DeepCORO_df_angle_object_dicom_2017-2021.csv')
### Drop Unnamed columns
df_angle_object = df_angle_object.loc[:, ~df_angle_object.columns.str.contains('^Unnamed')]


In [ ]:
# Perform left merge
df_merged_angle_object = pd.merge(df_merged, df_angle_object, on='path', how='left')


In [ ]:
display(df_merged_angle_object.object_value.value_counts())

In [ ]:
display(df_merged_angle_object.contains_lad.value_counts())
display(df_merged_angle_object.contains_lcx.value_counts())
display(df_merged_angle_object.contains_rca.value_counts())

In [ ]:
df_filtered = df_merged_angle_object[
    (df_merged_angle_object['contains_lad'] == True) | 
    (df_merged_angle_object['contains_lcx'] == True) | 
    (df_merged_angle_object['contains_rca'] == True)
]

In [ ]:
#1425 examens avec IFR dans IVA, LCX ou RCA
display(df_f.grou

In [ ]:
df_f.to_csv('data/df_f.csv', index=False)
df_merged_angle_object.to_csv('data/df_merged_angle_object_IFR.csv', index=False)

Sure, here's a markdown table based on the labels' definitions:

| Label | Name         | 'primary' range | 'secondary' range |
|-------|--------------|-----------------|-------------------|
| 0     | RAO Cranial  | -60 to -15      | 15 to 50          |
| 1     | AP Cranial   | -15 to 15       | 15 to 50          |
| 2     | LAO Cranial  | 15 to 60        | 15 to 50          |
| 3     | RAO Straight | -60 to -15      | -15 to 15         |
| 4     | AP           | -15 to 15       | -15 to 15         |
| 5     | LAO Straight | 15 to 60        | -15 to 15         |
| 6     | RAO Caudal   | -60 to -15      | -50 to -15        |
| 7     | AP Caudal    | -15 to 15       | -50 to -15        |
| 8     | LAO Caudal   | 15 to 60        | -50 to -15        |
| 9     | LAO Lateral  | -110 to -70     | -15 to 15         |
| 10    | RAO Lateral  | 70 to 110       | -15 to 15         |
| 11    | Other        | Not applicable  | Not applicable    |

Note: The ranges in the 'primary' and 'secondary' columns are inclusive.

## Generate videos

In [2]:
import pandas as pd
import numpy as np
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from downloadAvi import extract_avi_metadata as avi_meta

/tmp/ipykernel_223997/3718068340.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


1 /media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.1839.20171026130739666.2.1.dcm
Error in pixel data /media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.1839.20171026130739666.2.1.dcm
2 /media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.1467.20171026122633739.4.4.dcm
3 /media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.9500.20171026122706279.1.8.dcm
4 /media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.7724.20171026122739520.2.12.dcm
5 /media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.3358.20171026122817989.3.16.dcm
6 /media/data1

In [3]:
df_f = pd.read_csv('data/df_f.csv')
#df_merged_angle_object = pd.read_csv('data/df_merged_angle_object_IFR.csv')

In [4]:
avi_meta.extract_avi_and_metadata("data/df_f.csv",  data_type="ANGIO", destinationFolder="deepIFR/", dataFolder="data/")

0it [00:00, ?it/s]

12it [00:03,  3.84it/s]
/volume/DicomVideoProcessing/downloadAvi/extract_avi_metadata.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if fps_col_names:


,brand,sex,FPS,NumberOfFrames,date,study_time,series_time,birthdate,color_format,mrn,StudyID,StudyInstanceUID,SeriesInstanceUID,dicom_path,FileName,primary_angle,secondary_angle,width,height,pixel_spacing,distance_source_to_detector,distance_source_to_patient,estimated_radiographic_magnification_factor,table_motion,radiation_setting,image_pixel_spacing,Split
0,,,7,51,20171026,075609.000000,082624.601000,19560405,MONOCHROME2,102691316522124012470150158280608329264,'1.2.392.200036.9116.2145.20171026130754041.1.12',2.16.124.113611.1.118.1.1.4776839,'1.2.392.200036.9116.1467.20171026122633739.4.4',/media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.1467.20171026122633739.4.4.dcm,deepIFR/2.16.124.113611.1.118.1.1.4776839_1.2.392.200036.9116.1467.20171026122633739.4.4.dcm.avi,27.0,0.0,512,512,"[0.184928, 0.184928]",990.0,637.4598,1.553039,STATIC,GR,"[0.2872, 0.2872]",inference
1,,,7,54,20171026,075609.000000,082657.154000,19560405,MONOCHROME2,102691316522124012470150158280608329264,'1.2.392.200036.9116.6281.20171026130804437.2.13',2.16.124.113611.1.118.1.1.4776839,'1.2.392.200036.9116.9500.20171026122706279.1.8',/media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.9500.20171026122706279.1.8.dcm,deepIFR/2.16.124.113611.1.118.1.1.4776839_1.2.392.200036.9116.9500.20171026122706279.1.8.dcm.avi,-24.0,-26.0,512,512,"[0.176452, 0.176452]",1110.0,681.9703,1.627637,STATIC,GR,"[0.2872, 0.2872]",inference
2,,,7,59,20171026,075609.000000,082729.988000,19560405,MONOCHROME2,102691316522124012470150158280608329264,'1.2.392.200036.9116.8827.20171026130813683.8.14',2.16.124.113611.1.118.1.1.4776839,'1.2.392.200036.9116.7724.20171026122739520.2.12',/media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.7724.20171026122739520.2.12.dcm,deepIFR/2.16.124.113611.1.118.1.1.4776839_1.2.392.200036.9116.7724.20171026122739520.2.12.dcm.avi,-18.0,40.0,512,512,"[0.171441, 0.171441]",1130.0,674.5429,1.675208,STATIC,GR,"[0.2872, 0.2872]",inference
3,,,7,53,20171026,075609.000000,082808.613000,19560405,MONOCHROME2,102691316522124012470150158280608329264,'1.2.392.200036.9116.1961.20171026130823853.2.15',2.16.124.113611.1.118.1.1.4776839,'1.2.392.200036.9116.3358.20171026122817989.3.16',/media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.3358.20171026122817989.3.16.dcm,deepIFR/2.16.124.113611.1.118.1.1.4776839_1.2.392.200036.9116.3358.20171026122817989.3.16.dcm.avi,36.0,24.0,512,512,"[0.176576, 0.176576]",1060.0,651.7086,1.626494,STATIC,GR,"[0.2872, 0.2872]",inference
4,,,7,41,20171026,075609.000000,082849.829000,19560405,MONOCHROME2,102691316522124012470150158280608329264,'1.2.392.200036.9116.1491.20171026130832936.1.16',2.16.124.113611.1.118.1.1.4776839,'1.2.392.200036.9116.7464.20171026122857190.1.20',/media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.7464.20171026122857190.1.20.dcm,deepIFR/2.16.124.113611.1.118.1.1.4776839_1.2.392.200036.9116.7464.20171026122857190.1.20.dcm.avi,24.0,-44.0,512,512,"[0.156295, 0.156295]",1250.0,680.2548,1.837547,STATIC,GR,"[0.2872, 0.2872]",inference
5,,,7,54,20171026,075609.000000,082921.402000,19560405,MONOCHROME2,102691316522124012470150158280608329264,'1.2.392.200036.9116.3995.20171026130841360.2.17',2.16.124.113611.1.118.1.1.4776839,'1.2.392.200036.9116.2145.20171026122930291.1.24',/media/data1/ravram/DeepCORO/2017/102691316522124012470150158280608329264/2.16.124.113611.1.118.1.1.4776839/1.2.392.200036.9116.2145.20171026122930291.1.24.dcm,deepIFR/2.16.124.113611.1.118.1.1.4776839_1.2.392.200036.9116.2145.20171026122930291.1.24.dcm.avi,43.0,-24.0,512,512,"[0.174198, 0.174198]",1140.0,691.4541,1.648700,STATIC,GR,"[0.2872, 0.2872]",inference
6,,,7,49,20171026,

In [5]:
import pydicom as dicom
from pydicom.uid import UID, generate_uid
dataset = dicom.dcmread(df_f.path.iloc[1], force=True)

In [6]:
display(dataset)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 182
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: X-Ray Angiographic Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.392.200036.9116.2145.20171026130754041.1.12
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-1.4.38'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['DERIVED', 'SECONDARY', 'SINGLE PLANE', 'DA', 'ACQ']
(0008, 0013) Instance Creation Time              TM: '075609.104000'
(0008, 0014) Instance Creator UID                UI: 9.233395599149415911703030782493510826184
(0008, 0016) SOP Class UID        